In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd

In [2]:
conf = SparkConf().setAppName('spark_pd').setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
sc

<SparkContext master=local appName=spark_pd>

In [4]:
df = spark.read.json('/usr/local/spark/examples/src/main/resources/people.json')

In [5]:
df

DataFrame[age: bigint, name: string]

In [6]:
df.show(3)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
# spark, df are from the previous example
# Print the schema in a tree format
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
df.select(df['name'], df['age'] + 1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [8]:
df.filter(df['age'] > 21).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [9]:
df.groupBy('age').count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



Temporary views in Spark SQL are session-scoped and will disappear if the session that creates it terminates.

In [10]:
df.createOrReplaceTempView("people")
sqlDF = spark.sql("select * from people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



Global temporary view is tied to a system preserved database global_temp, and we must use the qualified name to refer it, e.g. SELECT * FROM global_temp.view1

In [12]:
# Register the DataFrame as a global temporary view
df.createGlobalTempView("people")

In [13]:
spark.sql("select * from global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [14]:
# Global temporary view is cross-session
spark.newSession().sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



Spark SQL支持两种将现有RDD转换为数据集的方法。 第一种方法使用反射来推断包含特定对象类型的RDD的架构。 这种基于反射的方法可以使代码更简洁，并且当您在编写Spark应用程序时已经了解架构时，可以很好地工作。

创建数据集的第二种方法是通过编程界面，该界面允许您构造模式，然后将其应用于现有的RDD。 尽管此方法较为冗长，但可以在运行时才知道列及其类型的情况下构造数据集。

## Inferring the Schema Using Reflection

Spark SQL can convert an RDD of Row objects to a DataFrame, inferring the datatypes

Spark SQL可以将Row对象的RDD转换为DataFrame，从而推断数据类型

In [11]:
from pyspark.sql import Row

In [12]:
lines = sc.textFile("/usr/local/spark/examples/src/main/resources/people.txt")
lines

/usr/local/spark/examples/src/main/resources/people.txt MapPartitionsRDD[24] at textFile at NativeMethodAccessorImpl.java:0

In [13]:
parts = lines.map(lambda l: l.split(','))
parts.take(1)

[['Michael', ' 29']]

In [14]:
# 将键/值对列表作为kwargs传递给Row类来构造行
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))
people.take(1)

[Row(age=29, name='Michael')]

In [15]:
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")

In [16]:
# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("select name from people where age between 13 and 19")
teenagers.show()

+------+
|  name|
+------+
|Justin|
+------+



In [20]:
teenagers.rdd

MapPartitionsRDD[47] at javaToPython at NativeMethodAccessorImpl.java:0

In [17]:
# The results of SQL queries are Dataframe objects.
# rdd returns the content as an :class:`pyspark.RDD` of :class:`Row`.
teen_names = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()
for name in teen_names:
    print(name)

Name: Justin


## Programmatically Specifying the Schema

1. 从原始RDD创建元组或列表的RDD；
2. 在第1步中创建的RDD中，创建一个由StructType表示的模式来匹配元组或列表的结构。
3. 通过SparkSession提供的createDataFrame方法将架构应用于RDD。

NullType, StringType, BinaryType, BooleanType, DateType, TimestampType, DecimalType, DoubleType
FloatType, ByteType, IntegerType 32, LongType 64, ShortType 16, ArrayType, MapType

In [18]:
from pyspark.sql.types import *

# sc = spark.sparkContext
lines = sc.textFile("/usr/local/spark/examples/src/main/resources/people.txt")
parts = lines.map(lambda l: l.split(','))
#  Each line is converted to a tuple.
people = parts.map(lambda p: (p[0], int(p[1].strip())))


In [31]:
# The schema is encoded in a string.
schemaString = "name age"

In [19]:
# 字段名 字段类型 nullable
fields = [StructField('name', StringType(), True), StructField('age', ShortType(), True)]
schema = StructType(fields)

In [20]:
# Apply the schema to the RDD.
shemaPeople = spark.createDataFrame(people, schema)

In [21]:
shemaPeople.createOrReplaceTempView("people1")

In [22]:
shemaPeople.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [23]:
spark.sql("select name from people1").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



# Generic Load/Save Functions

In [24]:
df = spark.read.load("/usr/local/spark/examples/src/main/resources/users.parquet")
df.select("name", "favorite_color").show()

+------+--------------+
|  name|favorite_color|
+------+--------------+
|Alyssa|          null|
|   Ben|           red|
+------+--------------+



In [25]:
df.select("name", "favorite_color").write.save("data/namesAndFavColors.parquet")

In [26]:
# 指定读取和保存 文件的格式
df = spark.read.load("/usr/local/spark/examples/src/main/resources/people.json", format="json")
df.select("name", "age").write.save("data/nameAndAges.parquet", format='parquet')

In [27]:
df = spark.read.csv("/usr/local/spark/examples/src/main/resources/people.csv", sep=';', inferSchema=True, header=True)
df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [45]:
# 读取csv
df = spark.read.load("/usr/local/spark/examples/src/main/resources/people.csv", 
                    format="csv", sep=";", inferSchema=True, header=True)
df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [28]:
# Run SQL on files directly
df = spark.sql("select * from parquet.`/usr/local/spark/examples/src/main/resources/users.parquet`")
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



### Bucketing, Sorting and Partitioning

In [29]:

# Bucketing and sorting are applicable only to persistent tables
df = spark.read.load("/usr/local/spark/examples/src/main/resources/people.json", format="json")
df.write.bucketBy(42, "name").sortBy('age').saveAsTable("people_bucketed")

- saveAsTable(name, format=None, mode=None, partitionBy=None, **options)

    Saves the content of the DataFrame as the specified table.

- save(path=None, format=None, mode=None, partitionBy=None, **options)

    Saves the contents of the DataFrame to a data source.

In [30]:
spark.sql("select * from people_bucketed").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [33]:
#  partitioning can be used with both save and saveAsTable when using the Dataset APIs
df = spark.sql("select * from parquet.`/usr/local/spark/examples/src/main/resources/users.parquet`")
df.write.partitionBy("favorite_color").format("parquet").save("data/namesPartByColor.parquet")

## Parquet Files

列存储格式


In [34]:
peopleDF = spark.read.json("/usr/local/spark/examples/src/main/resources/people.json")
# DataFrames can be saved as Parquet files, maintaining the schema information.
peopleDF.write.parquet("data/people.parquet")

In [35]:
# Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a DataFrame.
parquetFile = spark.read.parquet("data/people.parquet")

In [36]:
parquetFile.createOrReplaceTempView("parquetFile")
teenagers = spark.sql("SELECT name FROM parquetFile WHERE age >= 13 AND age <= 19")
teenagers.show()

+------+
|  name|
+------+
|Justin|
+------+



In [37]:
## Schema Merging
from pyspark.sql import Row

In [38]:
squaresDF = spark.createDataFrame(sc.parallelize(range(1, 6))
                                  .map(lambda i: Row(single=i, doule= i ** 2)))

In [39]:
squaresDF.write.parquet("data/test_table/key=1")

In [40]:
# Create another DataFrame in a new partition directory,
# adding a new column and dropping an existing column
cubesDF = spark.createDataFrame(sc.parallelize(range(6, 11))
                                .map(lambda i: Row(single=i, triple=i ** 3)))

In [41]:
cubesDF.write.parquet('data/test_table/key=2')

- option(key, value)

    Adds an input option for the underlying data source.

In [42]:
# Read the partitioned table
mergedDF = spark.read.option("mergeSchema", "true").parquet("data/test_table")
mergedDF.printSchema()

root
 |-- doule: long (nullable = true)
 |-- single: long (nullable = true)
 |-- triple: long (nullable = true)
 |-- key: integer (nullable = true)



In [43]:
mergedDF.show()

+-----+------+------+---+
|doule|single|triple|key|
+-----+------+------+---+
| null|     6|   216|  2|
| null|     7|   343|  2|
| null|     8|   512|  2|
| null|     9|   729|  2|
| null|    10|  1000|  2|
|    1|     1|  null|  1|
|    4|     2|  null|  1|
|    9|     3|  null|  1|
|   16|     4|  null|  1|
|   25|     5|  null|  1|
+-----+------+------+---+



In [44]:
# Metadata Refreshing
spark.catalog.refreshTable("my_table")

AnalysisException: "Table or view 'my_table' not found in database 'default';"

In [45]:
spark.sql("show tables").show()

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
| default|people_bucketed|      false|
|        |    parquetfile|       true|
|        |         people|       true|
|        |        people1|       true|
+--------+---------------+-----------+



In [46]:
spark.conf.set("spark.sql.orc.enableVectorizedReader", "true")

In [66]:
# A JSON dataset is pointed to by path.
# The path can be either a single text file or a directory storing text files
path = "/usr/local/spark/examples/src/main/resources/people.json"
peopleDF = spark.read.json(path)

In [67]:
# The inferred schema can be visualized using the printSchema() method
peopleDF.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [68]:
# Creates a temporary view using the DataFrame
peopleDF.createOrReplaceTempView("people")

In [47]:
# SQL statements can be run by using the sql methods provided by spark
teenagerNamesDF = spark.sql("SELECT name FROM people WHERE age BETWEEN 13 AND 19")
teenagerNamesDF.show()

+------+
|  name|
+------+
|Justin|
+------+



In [48]:
import json
d1 = {'name':1, 'age':2}
josn_str = json.dumps(d1)
josn_str

'{"name": 1, "age": 2}'

In [49]:
# Alternatively, a DataFrame can be created for a JSON dataset represented by
# an RDD[String] storing one JSON object per string
# jsonStrings = ['{"name":"Yin","address":{"city":"Columbus","state":"Ohio"}}']
otherPeopleRDD = sc.parallelize([josn_str])

In [50]:
otherPeople = spark.read.json(otherPeopleRDD)
otherPeople.show()

+---+----+
|age|name|
+---+----+
|  2|   1|
+---+----+



## Hive Tables

In [27]:
sc.stop()

In [51]:
from os.path import expanduser, join, abspath

from pyspark.sql import SparkSession
from pyspark.sql import Row


In [54]:
warehouse_location = abspath('spark-warehouse')

In [55]:
warehouse_location

'/mnt/data1/workspace/data_analysis_mining/Python_Spark/spark_tutorial/SQL/spark-warehouse'

In [61]:
spark.stop()

In [62]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

In [63]:
spark

In [65]:
# spark is an existing SparkSession
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")
# CREATE TABLE src(id int) USING hive OPTIONS(fileFormat 'parquet')
spark.sql("load data local inpath '/usr/local/spark/examples/src/main/resources/kv1.txt' into table src")

DataFrame[]

In [66]:
spark.sql("select count(*) from src").show()

+--------+
|count(1)|
+--------+
|     500|
+--------+



In [67]:
results = spark.sql("select key, value from src where key < 10 order by key")
results.show()

+---+-----+
|key|value|
+---+-----+
|  0|val_0|
|  0|val_0|
|  0|val_0|
|  2|val_2|
|  4|val_4|
|  5|val_5|
|  5|val_5|
|  5|val_5|
|  8|val_8|
|  9|val_9|
+---+-----+



In [68]:
# The items in DataFrames are of type Row, which allows you to access each column by ordinal.
stringDS = results.rdd.map(lambda row: "Key: {}, Value: {}".format(row.key, row.value))
stringDS.take(2)

['Key: 0, Value: val_0', 'Key: 0, Value: val_0']

In [69]:
for record in stringDS.collect():
    print(record)

Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 2, Value: val_2
Key: 4, Value: val_4
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 8, Value: val_8
Key: 9, Value: val_9


In [70]:
# You can also use DataFrames to create temporary views within a SparkSession.
Record = Row('key', 'Value')
recordsDF = spark.createDataFrame([Record(i, "val_" + str(i)) for i in range(1, 101)])
recordsDF

DataFrame[key: bigint, Value: string]

In [71]:
recordsDF.createOrReplaceTempView("records")
# Queries can then join DataFrame data with data stored in Hive.
spark.sql("SELECT * FROM records r JOIN src s ON r.key = s.key").show()

+---+------+---+------+
|key| Value|key| value|
+---+------+---+------+
|  2| val_2|  2| val_2|
|  4| val_4|  4| val_4|
|  5| val_5|  5| val_5|
|  5| val_5|  5| val_5|
|  5| val_5|  5| val_5|
|  8| val_8|  8| val_8|
|  9| val_9|  9| val_9|
| 10|val_10| 10|val_10|
| 11|val_11| 11|val_11|
| 12|val_12| 12|val_12|
| 12|val_12| 12|val_12|
| 15|val_15| 15|val_15|
| 15|val_15| 15|val_15|
| 17|val_17| 17|val_17|
| 18|val_18| 18|val_18|
| 18|val_18| 18|val_18|
| 19|val_19| 19|val_19|
| 20|val_20| 20|val_20|
| 24|val_24| 24|val_24|
| 24|val_24| 24|val_24|
+---+------+---+------+
only showing top 20 rows



## Apache Avro Data

spark-submit --packages org.apache.spark:spark-avro_2.12:2.4.6

spark-shell --packages org.apache.spark:spark-avro_2.12:2.4.6

In [102]:
df = spark.read.format("avro").load("/usr/local/spark/examples/src/main/resources/users.avro")
df.select("name", "favorite_color").write.format("avro").save("namesAndFavColors.avro")

AnalysisException: 'Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of "Apache Avro Data Source Guide".;'

# 读写MySQL

下载MySQL的JDBC驱动程序，比如mysql-connector-java-5.1.40.tar.gz, 解压提取jar驱动

把该驱动程序拷贝到spark的安装目录” /usr/local/spark/jars”下

https://spark.apache.org/docs/2.4.6/sql-data-sources-jdbc.html

## 读

In [72]:
user = "root"
passwd = "62300313.a"
host = "localhost"
db_name = "db1"
table_name = "emp"

In [73]:
spark.read.jdbc?

Signature:
spark.read.jdbc(
    url,
    table,
    column=None,
    lowerBound=None,
    upperBound=None,
    numPartitions=None,
    predicates=None,
    properties=None,
)
Docstring:
Construct a :class:`DataFrame` representing the database table named ``table``
accessible via JDBC URL ``url`` and connection ``properties``.

Partitions of the table will be retrieved in parallel if either ``column`` or
``predicates`` is specified. ``lowerBound`, ``upperBound`` and ``numPartitions``
is needed when ``column`` is specified.

If both ``column`` and ``predicates`` are specified, ``column`` will be used.

.. note:: Don't create too many partitions in parallel on a large cluster;
    otherwise Spark might crash your external database systems.

:param url: a JDBC URL of the form ``jdbc:subprotocol:subname``
:param table: the name of the table
:param column: the name of a column of numeric, date, or timestamp type
               that will be used for partitioning;
               if this parame

In [74]:
jdbcDF = spark.read.jdbc(
    url=f"jdbc:mysql://{host}:3306/{db_name}",
    table=f"{table_name}",
    properties={'user': f"{user}", "password":f"{passwd}"}
)

In [ ]:
# jdbcDF =  spark.read.format('jdbc')\
#             .option('driver', "com.mysql.jdbc.Driver")\
#             .option('url', f"jdbc:mysql://{host}:3306/{db_name}")\
#             .option('dbtable', f"{table_name}")\
#             .option("user", f"{user}")\
#             .option('password', f"{passwd}")\
#             .load()

In [75]:
jdbcDF.printSchema()

root
 |-- id: long (nullable = true)
 |-- empno: long (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: long (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: decimal(7,2) (nullable = true)
 |-- comm: decimal(7,2) (nullable = true)
 |-- deptno: long (nullable = true)



In [8]:
# jdbcDF.count()

10000000

In [76]:
jdbcDF.filter("deptno = 100").count()

998703

## 写

In [81]:
sc = spark.sparkContext

In [77]:
from pyspark.sql.types import *

In [78]:
table_name = "stu"

In [79]:
schema = StructType(fields=[
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True)
])

In [82]:
student = sc.parallelize([(10001, 's1'), (10002, 's2')])

In [83]:
df1 = spark.createDataFrame(student, schema=schema)
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- favorite_color: string (nullable = true)
 |-- favorite_numbers: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [24]:
df.write.jdbc?

Signature: df.write.jdbc(url, table, mode=None, properties=None)
Docstring:
Saves the content of the :class:`DataFrame` to an external database table via JDBC.

.. note:: Don't create too many partitions in parallel on a large cluster;
    otherwise Spark might crash your external database systems.

:param url: a JDBC URL of the form ``jdbc:subprotocol:subname``
:param table: Name of the table in the external database.
:param mode: specifies the behavior of the save operation when data already exists.

    * ``append``: Append contents of this :class:`DataFrame` to existing data.
    * ``overwrite``: Overwrite existing data.
    * ``ignore``: Silently ignore this operation if data already exists.
    * ``error`` or ``errorifexists`` (default case): Throw an exception if data already                 exists.
:param properties: a dictionary of JDBC database connection arguments. Normally at
                   least properties "user" and "password" with their corresponding values.
        

In [26]:
df.write.format('jdbc')\
            .option('url', f"jdbc:mysql://{host}:3306/{db_name}")\
            .option('dbtable', f"{table_name}")\
            .option("user", f"{user}")\
            .option('password', f"{passwd}")\
            .save(mode='append')

In [84]:
student = sc.parallelize([(10005, 's1'), (10006, 's2')])
df2 = spark.createDataFrame(student, schema=schema)

In [85]:
df2.write.jdbc(url=f"jdbc:mysql://{host}:3306/{db_name}",
              table=f"{table_name}",
              mode='append',
              properties={'user': f"{user}", 
                          "password":f"{passwd}", 
                          "driver":"com.mysql.jdbc.Driver"}
              )

```
MariaDB [db1]> select * from stu;
+-------+--------+
| id    | name   |
+-------+--------+
|     1 | 李思   |
|     2 | aaa    |
|     3 | bbb    |
|     2 | aaa    |
|     3 | bbb    |
| 30001 | s1     |
| 30002 | s2     |
| 10001 | s1     |
| 10002 | s2     |
+-------+--------+

```